In [21]:
import os
import torch
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import Tensor

# Imports de las funciones creadas para este programa
from models.constant_diffusivity import ConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [22]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data\linear_homogeneous\linear_homogeneous.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results\linear_homogeneous')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results\linear_homogeneous\lineal_homogeneous_prueba')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder successfully created at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\linear_homogeneous
Folder successfully created at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\linear_homogeneous\lineal_homogeneous_prueba


In [23]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\data\linear_homogeneous\linear_homogeneous.pkl


In [24]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [25]:
# Tratamiento de los datos para dividirlos en train y test
X_train = Tensor(torch.tensor(dataset['X_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, order=0)
y_train = Tensor(torch.tensor(dataset['y_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, order=0)
K_train = Tensor(torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, order=0)

X_val = Tensor(torch.tensor(dataset['X_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, order=0)
y_val = Tensor(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, order=0)
K_val = Tensor(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, order=0)

X_np = X_train.values
y_np = y_train.values
K_np = K_train.values

X_train_np, X_test_np, y_train_np, y_test_np, K_train_np, K_test_np = train_test_split(X_np, y_np, K_np, test_size=0.2, random_state=42)

X_train = Tensor(X_train_np, space_dimension=X_train.space_dim, order=X_train.order)
X_test = Tensor(X_test_np, space_dimension=X_train.space_dim, order=X_train.order)

y_train = Tensor(y_train_np, space_dimension=y_train.space_dim, order=y_train.order)
y_test = Tensor(y_test_np, space_dimension=y_train.space_dim, order=y_train.order)

K_train = Tensor(K_train_np, space_dimension=K_train.space_dim, order=K_train.order)
K_test = Tensor(K_test_np, space_dimension=K_train.space_dim, order=K_train.order)

In [26]:
# Arquitectura del modelo
input_shape = X_train.values[0].shape  # [1, 10, 8]
hidden1_dim = 100
hidden2_dim = 100
output_shape = y_train.values[0].shape  # [1, 10, 10]

In [27]:
# Se carga el modelo y el optimizador
model = ConstantDiffusivityNeuralNetwork(input_shape, hidden1_dim, hidden2_dim, output_shape)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [28]:
# Parametros de entrenamiento
start_epoch = 0
n_epochs = 100

batch_size = 64

In [29]:
train_loop(model, optimizer,
           X_train, y_train, X_test, y_test, D=D, 
           start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH
        )